In [1]:
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

from carla.data.causal_model import CausalModel
import shap
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [ ]:
# Load Data 
X,y = shap.datasets.adult()
X_display,y_display = shap.datasets.adult(display=True) # human readable feature values
#TODO use ordinal Encoding ? 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)


In [3]:
#Model

In [4]:
scm = CausalModel("census")
dataset = scm.generate_dataset(10000)

display(dataset.df)
#scm

Census Model Initiate
Structural Equation Finished
Noise Distribution Finished
({'x1': <function census_model.<locals>.<lambda> at 0x7fa89a8a4e60>, 'x2': <function census_model.<locals>.<lambda> at 0x7fa89a8a4ef0>, 'x3': <function census_model.<locals>.<lambda> at 0x7fa89a8a4f80>, 'x4': <function census_model.<locals>.<lambda> at 0x7fa89a8d9050>, 'x5': <function census_model.<locals>.<lambda> at 0x7fa89a8d90e0>, 'x6': <function census_model.<locals>.<lambda> at 0x7fa89a8d9170>, 'x7': <function census_model.<locals>.<lambda> at 0x7fa89a8d9200>, 'x8': <function census_model.<locals>.<lambda> at 0x7fa89a8d9290>, 'x9': <function census_model.<locals>.<lambda> at 0x7fa89a8d9320>}, {'x1': <function census_model.<locals>.<lambda> at 0x7fa89a8a4e60>, 'x2': <function census_model.<locals>.<lambda> at 0x7fa89a8a4ef0>, 'x3': <function census_model.<locals>.<lambda> at 0x7fa89a8a4f80>, 'x4': <function census_model.<locals>.<lambda> at 0x7fa89a8d9050>, 'x5': <function census_model.<locals>.<lambda>

,label,x1,x2,x3,x7,x8,x4,x5,x6,x9
0,1.0,2.254946,0.698706,2.731092,-2.236175,1.664299,-0.490102,0.925852,0.243611,0.447205
1,0.0,0.388335,-0.853614,-1.836453,-1.405027,-0.535627,1.464407,-1.847255,-0.175946,-0.188734
2,1.0,1.673600,-0.480518,-0.610168,-0.249641,-0.152340,1.102873,1.043811,-0.172251,0.382483
3,0.0,0.857235,0.261332,-1.121407,0.493967,-0.748096,0.345517,-0.692395,1.424461,-2.037972
4,0.0,-0.779046,0.375737,1.108119,1.052808,1.162800,-0.736217,-1.176551,-0.396180,-0.854447
...,...,...,...,...,...,...,...,...,...,...
9995,0.0,-1.551671,0.466290,-0.374929,-0.677533,0.360639,1.154560,-0.310784,-0.189896,1.269068
9996,1.0,1.675497,-3.074944,-0.814784,0.280804,-1.024606,1.507610,0.990465,-0.562605,2.258548
9997,0.0,-3.703923,-0.490037,1.062704,0.150499,0.168849,-1.110003,0.630036,-0.888076,2.375251
9998,0.0,-3.032812,-1.687168,0.913443,1.298857,-0.535102,-0.873068,0.249871,0.264209,-1.756526


In [5]:
from carla.models.catalog import MLModelCatalog


training_params = {"lr": 0.01, "epochs": 10, "batch_size": 16, "hidden_size": [18, 9, 3]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)

balance on test set 0.44426666666666664, balance on test set 0.4436
Epoch 0/9
----------
train Loss: 0.4027 Acc: 0.8252

test Loss: 0.3625 Acc: 0.8456

Epoch 1/9
----------
train Loss: 0.3702 Acc: 0.8364

test Loss: 0.3620 Acc: 0.8488

Epoch 2/9
----------
train Loss: 0.3682 Acc: 0.8321

test Loss: 0.3519 Acc: 0.8472

Epoch 3/9
----------
train Loss: 0.3663 Acc: 0.8347

test Loss: 0.3605 Acc: 0.8448

Epoch 4/9
----------
train Loss: 0.3629 Acc: 0.8399

test Loss: 0.3567 Acc: 0.8444

Epoch 5/9
----------
train Loss: 0.3628 Acc: 0.8376

test Loss: 0.3503 Acc: 0.8480

Epoch 6/9
----------
train Loss: 0.3628 Acc: 0.8379

test Loss: 0.3556 Acc: 0.8408

Epoch 7/9
----------
train Loss: 0.3612 Acc: 0.8353

test Loss: 0.3711 Acc: 0.8352

Epoch 8/9
----------
train Loss: 0.3608 Acc: 0.8364

test Loss: 0.3547 Acc: 0.8480

Epoch 9/9
----------
train Loss: 0.3613 Acc: 0.8361

test Loss: 0.3525 Acc: 0.8432



In [6]:
from carla.models.negative_instances import predict_negative_instances
from carla.recourse_methods.catalog.causal_recourse import (
    CausalRecourse,
    constraints,
    samplers,
)


In [7]:
# get factuals
factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:5]

In [8]:
hyperparams = {
    "optimization_approach": "brute_force",
    "num_samples": 10,
    "scm": scm,
    "constraint_handle": constraints.point_constraint,
    "sampler_handle": samplers.sample_true_m0,
}
cfs = CausalRecourse(ml_model, hyperparams).get_counterfactuals(test_factual)

display(cfs)

: 

: 